## Setup

In [75]:
!pip install openai==1.2.0

In [90]:
import os
import pandas as pd
import json

In [142]:
import openai
from openai import OpenAI
client = OpenAI(
  api_key="bks-3f6fd9130cb4ba794b33bb22b1829f2fad3e0fbfc576cf78",
  base_url="https://api.trybricks.ai/api/providers/openai/v1",
)
def get_completion(prompt, model = "gpt-3.5-turbo", temperature =0):
  messages = [{"role":"user", "content": prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature =temperature # degree of expiration (randomness) (0 same result - 1 creative)
  )
  return response.choices[0].message.content

In [78]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [183]:
names = os.listdir('/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript')[2:-1]
f_list = []
for n in names:
  path = '/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript/' + n
  f_list.append(pd.read_csv(path))

## EDA

In [ ]:
[len(f_list[i]) for i in range(len(f_list))] # number of turns

[73, 248, 150, 385, 163, 341, 322, 227, 302, 114, 348]

In [ ]:
[f_list[i]['Utterance end time (milliseconds)'][len(f_list[i])-1]/60000 for i in range(len(f_list))] # session length

[67.67616666666666,
 100.25683333333333,
 42.20483333333333,
 75.3735,
 65.7575,
 66.26966666666667,
 59.51383333333333,
 68.21333333333334,
 48.0325,
 70.76933333333334,
 90.33033333333333]

## Speaker identification

In [ ]:
idx = 9
temp = [f_list[idx]['Speaker'][i]+ " : " + f_list[idx]['Utterance'][i]  for i in range(len(f_list[idx]))]
d = ' '.join(temp) [:4097]

In [ ]:
prompt = f"""I will provide you a transcript of a tutor and a student.
There will to two speakers A and B. You task is to identify which is student and which is tutor. Make your output in a JSON format with A,B as key; student, tutor as value
""" + d
r = get_completion(prompt)

In [ ]:
a = json.loads(r)

In [ ]:
f_list[idx] = f_list[idx].replace({'Speaker': a})

In [ ]:
f_list[idx]

,Speaker,Utterance start time (milliseconds),Utterance end time (milliseconds),Utterance
0,student,8790,12710,You. That's all I wrote.
1,tutor,12790,13420,Okay.
2,student,37050,1077750,Or whatever. It.
3,C,1086120,1086870,Hello.
4,tutor,1087640,1088390,Hi.
...,...,...,...,...
109,tutor,4212020,4229590,Right. So four minus four would give us a zero...
110,C,4230680,4234150,"Well, that's good."
111,tutor,4235640,4239588,"Okay, good. That put our brains to work."
112,C,4239754,4241124,Indeed it did.


## Student expectation

In [143]:
idx = 9
temp = [f_list[idx]['Speaker'][i]+ " : " + f_list[idx]['Utterance'][i]  for i in range(len(f_list[idx]))]
d = ' '.join(temp) [:4097]

In [144]:
prompt = f"""I will provide you a transcript of a tutor and a student.
Find the student expectation of the session from the transcript.
Format your answer in Json format, with index of the expectation as key and content of expectation as values.
""" + d
r = get_completion(prompt)

In [146]:
json.loads(r)

{'1': 'Student expects the tutor to help with pronunciation and understanding of English',
 '2': 'Student expects the tutor to assist with their English classes',
 '3': 'Student expects the tutor to provide guidance and support in learning English',
 '4': 'Student expects the tutor to help with vocabulary and word meanings',
 '5': 'Student expects the tutor to assist with reading comprehension and understanding of a specific book'}

## Off topic

In [160]:
idx = 0
f_list[idx]['index'] = f_list[idx].index
temp = [f_list[idx]['Speaker'][i]+ "("+ str(f_list[idx]['index'][i]) + ")"+ " : " + f_list[idx]['Utterance'][i]  for i in range(len(f_list[idx]))]
d = ' '.join(temp) [:4097]

In [161]:
prompt = f"""I will provide you a transcript of a tutor and a student.
Identify the conversation that are unrelated to the learning process of the student.
Output the numbers in the parentheses
""" + d
r = get_completion(prompt)

In [162]:
r

"Conversation unrelated to the learning process of the student:\n(13) tutor: One of these days I need to visit some folks out in Colorado. I had a couple friends in there from the field work, get them to teach me to ski. A couple of the fire guys said they'd teach me if I visited them, but I don't know. We'll see. I feel like they'd all be really harsh teachers.\n(15) tutor: I will. Arnica cream. Arnica cream. That shit helps so much. It's like, just like. It smells good. Really helps with bruising. I only know that from, like, dancing a lot. Yeah. Anyways, glad it was a good time."

## Uncertain Tone

In [200]:
idx = 4
f_list[idx]['index'] = f_list[idx].index
temp = [f_list[idx]['Speaker'][i]+ "("+ str(f_list[idx]['index'][i]) + ")"+ " : " + f_list[idx]['Utterance'][i]  for i in range(len(f_list[idx]))]
d = ' '.join(temp) [:4097]

In [203]:
prompt = f"""I will provide you a transcript of a tutor and a student.
Did the student use uncertain tone during the session?
""" + d
r = get_completion(prompt)

In [204]:
r

'Yes, the student used an uncertain tone during the session. They used phrases such as "Okay, or whatever," "Let me look at the picture again," "Is that right?" and "I think it has an asymptote, probably x axis." These phrases indicate uncertainty and a lack of confidence in their answers.'

## KC

In [205]:
idx = 4
f_list[idx]['index'] = f_list[idx].index
temp = [f_list[idx]['Speaker'][i]+ "("+ str(f_list[idx]['index'][i]) + ")"+ " : " + f_list[idx]['Utterance'][i]  for i in range(len(f_list[idx]))]
d = ' '.join(temp) [:4097]

In [206]:
prompt = f""" A knowledge component is a description of a mental structure or process that a learner uses, alone or in combination with other knowledge components,
 to accomplish steps in a task or a problem
I will provide you a transcript of a tutor and a student.
Identify all the acadamic knowledge components mentioned in this transcript.
Format your answer in Json format, with index of the knowledge component as key and content of knowledge component as values.
""" + d
r = get_completion(prompt)

In [207]:
r

'{\n  "0": "mental structure or process",\n  "1": "assignment",\n  "2": "function",\n  "3": "positive",\n  "4": "negative",\n  "5": "increasing",\n  "6": "decreasing",\n  "7": "in behavior",\n  "8": "picture",\n  "9": "infinity",\n  "10": "real numbers",\n  "11": "cube root",\n  "12": "asymptote",\n  "13": "x axis"\n}'